# 1 Microsoft OmniParser

https://github.com/microsoft/OmniParser \
https://microsoft.github.io/OmniParser/ \
https://huggingface.co/microsoft/OmniParser \
\
OmniParser was the first model that I tested. My impressions after playing with its demo were grand (you can find it here: https://huggingface.co/spaces/microsoft/OmniParser); after providing it with a picture of Lidl's promo magazine, it was able to extract text from it with very high precision. Additionally, because it wasn't parsing any XML trees, output was very clear; as a result I got a list of extracted strings and a list of their bounding boxes. Using clustering or even simple algorithms to bind together close elements could provide us with a very clean data that later BERT model could easily label. \
Problems arise with the deployment of the model. As of today (12.11.24) I couldn't find any official data regarding the number of params of the model. Additionally, its deployment on a local machine is non-trivial: it's huggingface page doesn't hold all necessary files, so to run the model, you have to clone the github repo AND THEN download files from the huggingface page. And when I finally managed to do that, I found that this model can't be run without NVIDIA GPU drivers (I'm using an AMD GPU). \
In conclusion, I think that OmniParser could prove very useful to us, but we won't be able to deploy it to the mobile.